<a href="https://colab.research.google.com/github/yastiaisyah/DataSynthesis/blob/main/adversarial_vb_MSE_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from torchvision.datasets import MNIST
from torchvision import transforms

# Define data transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load MNIST dataset
mnist = MNIST(root='./data', train=True, transform=transform, download=True)

# Normalize the data to [0, 1] range
mnist.data = mnist.data.float() / 255.0

# Parameters
mb_size = 32
z_dim = 5
X_dim = mnist.data.size(1) * mnist.data.size(2)  # Flattened image dimensions
h_dim = 128
lr = 1e-3

# Create noise dimension
eps_dim = 10  # Dimension of the noise vector

# Encoder: q(z|x,eps)
Q = torch.nn.Sequential(
    torch.nn.Linear(X_dim + eps_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, z_dim)
)

# Decoder: p(x|z)
P = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, X_dim),
    torch.nn.Sigmoid()
)

# Discriminator: T(X, z)
T = torch.nn.Sequential(
    torch.nn.Linear(X_dim + z_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, 1)
)

def reset_grad():
    Q.zero_grad()
    P.zero_grad()
    T.zero_grad()


def sample_X(size):
    indices = np.random.randint(0, len(mnist), size)
    X = mnist.data[indices].view(size, -1).float()
    return Variable(X)


# Optimizers
Q_solver = optim.Adam(Q.parameters(), lr=lr)
P_solver = optim.Adam(P.parameters(), lr=lr)
T_solver = optim.Adam(T.parameters(), lr=lr)

# Initialize counter
cnt = 0
"""1000000"""

# Your training loop goes here
for it in range(100000):
    X = sample_X(mb_size)
    eps = Variable(torch.randn(mb_size, eps_dim))
    z = Variable(torch.randn(mb_size, z_dim))

    # Optimize VAE
    z_sample = Q(torch.cat([X, eps], 1))
    X_sample = P(z_sample)
    T_sample = T(torch.cat([X, z_sample], 1))

    disc = torch.mean(-T_sample)
    loglike = -nn.MSELoss()(X_sample, X) # Use MSELoss for binary cross-entropy

    elbo = -(disc + loglike)

    elbo.backward()
    Q_solver.step()
    P_solver.step()
    reset_grad()

    # Discriminator T(X, z)
    z_sample = Q(torch.cat([X, eps], 1))
    T_q = nn.Sigmoid()(T(torch.cat([X, z_sample], 1)))
    T_prior = nn.Sigmoid()(T(torch.cat([X, z], 1)))

    T_loss = -torch.mean(torch.log(T_q) + torch.log(1. - T_prior))

    T_loss.backward()
    T_solver.step()
    reset_grad()

    # Print and plot every now and then
    if it % 1000 == 0:
        print('Iter-{}; ELBO: {:.4}; T_loss: {:.4}'
              .format(it, -elbo.item(), -T_loss.item()))

        samples = P(z).data.numpy()[:16]

        fig = plt.figure(figsize=(4, 4))
        gs = gridspec.GridSpec(4, 4)
        gs.update(wspace=0.05, hspace=0.05)

        for i, sample in enumerate(samples):
            ax = plt.subplot(gs[i])
            plt.axis('off')
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_aspect('equal')
            plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

        if not os.path.exists('out/'):
            os.makedirs('out/')

        plt.savefig('out/{}.png'
                    .format(str(cnt).zfill(3)), bbox_inches='tight')
        cnt += 1
        plt.close(fig)

100%|██████████| 9912422/9912422 [00:00<00:00, 93281410.55it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 26925026.41it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 29992937.05it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4463572.81it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Iter-0; ELBO: -0.2218; T_loss: -1.387
Iter-1000; ELBO: -0.4044; T_loss: -1.16
Iter-2000; ELBO: -0.3174; T_loss: -1.258
Iter-3000; ELBO: -0.8561; T_loss: -0.8058
Iter-4000; ELBO: -0.05042; T_loss: -1.397
Iter-5000; ELBO: -0.1929; T_loss: -1.34
Iter-6000; ELBO: -0.1505; T_loss: -1.452
Iter-7000; ELBO: -0.06782; T_loss: -1.443
Iter-8000; ELBO: -0.2467; T_loss: -1.621
Iter-9000; ELBO: -0.1742; T_loss: -2.123
Iter-10000; ELBO: -0.7551; T_loss: -0.9851
Iter-11000; ELBO: -0.1226; T_loss: -1.689
Iter-12000; ELBO: -0.6476; T_loss: -1.027
Iter-13000; ELBO: -0.6222; T_loss: -0.9882
Iter-14000; ELBO: -0.4891; T_loss: -1.135
Iter-15000; ELBO: -0.2627; T_loss: -1.331
Iter-16000; ELBO: -0.5265; T_loss: -1.149
Iter-17000; ELBO: -0.1298; T_loss: -1.353
Iter-18000; ELBO: -0.06152; T_loss: -1.361
Iter-19000; ELBO: -0.1159; T_loss: -1.372
Iter-20000; ELBO: -0.4746; T_loss: -1.084
Iter-21000; ELBO: -1.804; T_loss: -1.145
Iter-22000; ELBO: -0.4785; T_loss: -1.135
Iter-23000; ELBO: 0.304; T_loss: -2.244
Iter